In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/data/train.csv')

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
#since we cannot add text to the missing data , we drop all the missing values
df = df.dropna()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18285 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      18285 non-null  int64 
 1   title   18285 non-null  object
 2   author  18285 non-null  object
 3   text    18285 non-null  object
 4   label   18285 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 857.1+ KB


In [8]:
# we seperate the label from the actual data
X = df.drop('label',axis = 1)

In [9]:
y = df['label']

In [11]:
tweets = X.copy()

In [12]:
tweets.reset_index(inplace = True)

In [15]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential


In [19]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()


In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [24]:
# Now, we have  to perform the data preprocessing steps.
list1 =[]# this is the list to contain all the text after cleaning .

In [25]:
for i in range (0,len(tweets)):
  cleaned = re.sub('[^a-zA-Z]',' ',tweets['title'][i])
  cleaned = cleaned.lower()
  cleaned = cleaned.split()
  cleaned = [ps.stem(word) for word in cleaned if not word in stopwords.words('english')]
  cleaned = ' '.join(cleaned)
  list1.append(cleaned)
               

In [26]:
from tensorflow.keras.preprocessing.text import one_hot

In [28]:

size = 5000
oh = [one_hot(words,size) for words in list1] # this converts all the words in the list1 to the numbers . you can look them below

In [29]:
oh 

[[2673, 1431, 2624, 4845, 2236, 1453, 1014, 3708, 1036, 1758],
 [1846, 2599, 2605, 491, 2918, 1398, 2421],
 [4147, 633, 4944, 714],
 [1021, 4982, 734, 4359, 1135, 616],
 [2793, 2918, 4985, 4391, 1644, 3491, 2918, 2834, 4884, 556],
 [3759,
  514,
  771,
  3586,
  4756,
  2186,
  666,
  4283,
  107,
  372,
  4919,
  664,
  869,
  3386,
  2421],
 [1618, 2839, 3944, 817, 4092, 1135, 4487, 30, 2978, 3649, 2811],
 [961, 4195, 2647, 3815, 388, 553, 2186, 3637, 2978, 3649, 2811],
 [750, 1113, 24, 340, 465, 4910, 3633, 3663, 2186, 4682],
 [1047, 4819, 3427, 402, 4695, 197, 2610, 2916],
 [2594, 2673, 707, 3332, 2811, 2681, 736, 3906, 4057, 3521, 704],
 [4359, 780, 2236, 4910, 2186, 388],
 [3664, 1416, 3644, 2398, 2933, 1463, 1714, 252, 4544],
 [688, 4505, 1813, 3330, 547, 192, 1682, 2978, 3649, 2811],
 [1572, 2650, 50, 4660, 1162, 2978, 3649, 2811],
 [4090, 965, 3831, 4912, 1930, 2995, 3786, 358, 2817, 399],
 [4289, 2586, 2599],
 [2667, 1202, 4289, 628, 2186, 2879, 3473, 2421],
 [3112, 4763, 260

In [30]:
# here list1 contains all the sentences. we want all the sentences ahould be of same size. lets take maximu words in a sentence equals to 15. that depends on choice
max_length = 15
from tensorflow.keras.preprocessing.sequence import pad_sequences
embedded_words = pad_sequences(oh,padding= 'pre',maxlen= max_length)


In [31]:
embedded_words

array([[   0,    0,    0, ..., 3708, 1036, 1758],
       [   0,    0,    0, ..., 2918, 1398, 2421],
       [   0,    0,    0, ...,  633, 4944,  714],
       ...,
       [   0,    0,    0, ..., 2978, 3649, 2811],
       [   0,    0,    0, ..., 4696, 2905, 3189],
       [   0,    0,    0, ...,  693, 4587,  883]], dtype=int32)

In [32]:
# its time to create the final model using word embeddings
embedded_vector_features = 50 # any number you can choose
model = Sequential()
model.add(Embedding(size,embedded_vector_features,input_length= max_length))
model.add(Dropout(0.2))
model.add(LSTM(120))
model.add(Dropout(0.2))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss='binary_crossentropy',optimizer ='adam',metrics=['accuracy'])



In [33]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 15, 50)            250000    
_________________________________________________________________
dropout (Dropout)            (None, 15, 50)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 120)               82080     
_________________________________________________________________
dropout_1 (Dropout)          (None, 120)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 121       
Total params: 332,201
Trainable params: 332,201
Non-trainable params: 0
_________________________________________________________________
None


In [34]:
X_final = np.array(embedded_words)
y_final = np.array(y)

In [35]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size = 0.3,random_state = 101)

In [36]:
model.fit(X_train,y_train,validation_data = (X_test,y_test),epochs = 10,batch_size = 45)

Epoch 1/10
285/285 [==============================] - 10s 24ms/step - loss: 0.2911 - accuracy: 0.8668 - val_loss: 0.1873 - val_accuracy: 0.9200
Epoch 2/10
285/285 [==============================] - 6s 22ms/step - loss: 0.1458 - accuracy: 0.9410 - val_loss: 0.1858 - val_accuracy: 0.9223
Epoch 3/10
285/285 [==============================] - 6s 22ms/step - loss: 0.1090 - accuracy: 0.9608 - val_loss: 0.2127 - val_accuracy: 0.9194
Epoch 4/10
285/285 [==============================] - 6s 22ms/step - loss: 0.0816 - accuracy: 0.9716 - val_loss: 0.2268 - val_accuracy: 0.9211
Epoch 5/10
285/285 [==============================] - 6s 22ms/step - loss: 0.0570 - accuracy: 0.9808 - val_loss: 0.2558 - val_accuracy: 0.9174
Epoch 6/10
285/285 [==============================] - 6s 22ms/step - loss: 0.0371 - accuracy: 0.9885 - val_loss: 0.3521 - val_accuracy: 0.9069
Epoch 7/10
285/285 [==============================] - 6s 22ms/step - loss: 0.0290 - accuracy: 0.9903 - val_loss: 0.3159 - val_accuracy: 0.913

In [37]:
y_pred = model.predict(X_test)


In [40]:
from sklearn.metrics import confusion_matrix,accuracy_score


In [44]:
accuracy_score(y_test, y_pred.round())

0.9074006562158221

In [49]:
y_pr = y_pred.round()

In [51]:
accuracy_score(y_test,y_pr)

0.9074006562158221

In [53]:
# our model has an accuracy of 90.7 percentage.